# Approximate Nearest Neighbors:

# Image Recommendation System via Collaborative Filtering

# ***Please read the instructions very carefully***
This is a modified version of the previous question and requires you to use an artificial nearest neighbors library

We suggest you to use one of the following:
- [ScaNN](https://github.com/google-research/google-research/tree/master/scann)
- [FAISS](https://github.com/facebookresearch/faiss)
- [Annoy](https://github.com/spotify/annoy.git)

1.   Assignment must be implemented in Python 3 only.
2.   You are allowed to use libraries for data preprocessing (numpy, pandas, nltk etc) and for evaluation metrics, data visualization (matplotlib etc.).
3.   You will be evaluated not just on the overall performance of the model and also on the experimentation with hyper parameters, data prepossessing techniques etc.
4.   ⚠️ The Assignment will be evaluated automatically. Please adhere to taking proper inputs from `config.csv` file. You can change your `config.csv` file to experiment with your code. But at the end, make sure that your outputs are corresponding to input values in `config.csv`
5.   Strict plagiarism checking will be done. An F will be awarded for plagiarism.

## About the Dataset
Behance is a community art website where users showcase and discover creative work. Each user is able to “appreciate” (equivalent to a “like” on Instagram or a “react” on Facebook) an image, indicating that they like the image. It is in the website’s best interests to show users pictures that they would like, to keep them engaged for longer. For this question, given a set of pictures that a user has already appreciated, you have to show them a new picture that they would like based on what similar users appreciated.


<br><br>
**The dataset has information of 1 million appreciates of 63,497 users on 178,788 items. The file Behance appreciate 1M has a triplet in each line in the form of (user id, item id, unix timestamp).**

**Task: Take the inputs from the config.csv file and output the recommendations for a particular person**
- Collaborative Filtering is a way to predict items to the user based on the the
user’s history and the history of similar users. The similarity between users can be quantified by the number of images that both the users appreciated.
- The images appreciated by a similar user would be the most suitable images to show a user. Since we can find the similarity between any two users, we would be able to find the “nearest” neighbours of any user, allowing us to use a KNN-based algorithm to recommend new images to a user.
- Since people do not like seeing pictures that they have seen already. Make sure that you do not recommend pictures that a user has appreciated already.
- Output the final response will be saved in the file named ```config['output_file']```.


**Output file format:**
Populate the output file with images that the user has not seen of the k most
similar users, in descending order of their similarity. Each line in the output
file should be a duplet in the form of (item id, user id), where the user id is the
id of the kth similar user. The order of the images corresponding to the same
similar user would not matter. The output file would look something like this:
```
item_id_1_of_1st_similar_user 1st_most_similar_user_id
item_id_2_of_1st_similar_user 1st_most_similar_user_id
item_id_3_of_1st_similar_user 1st_most_similar_user_id
...
item_id_1_of_2nd_similar_user 2nd_most_similar_user_id
item_id_2_of_2nd_similar_user 2nd_most_similar_user_id
item_id_3_of_2nd_similar_user 2nd_most_similar_user_id
...
item_id_1_of_kth_similar_user kth_most_similar_user_id
item_id_2_of_kth_similar_user kth_most_similar_user_id
item_id_3_of_kth_similar_user kth_most_similar_user_id
```

You may use any other recommendation system that you wish to use. However,
evaluation script will score your submission by measuring the similarity between
users with the number of common images they appreciated.
The dataset was extracted using Behance’s API as a part of the paper
“Vista: A visually, socially, and temporally-aware model for artistic
recommendation, RecSys, 2016”. Check out this [Google Drive folder](https://drive.google.com/drive/folders/0B9Ck8jw-TZUEc3NlMjVXdDlPU1k?resourcekey=0-6_8ykn0o4fLc5fuTEm91xA) for
more information about the dataset.


Have fun! The users are waiting to see new pictures!

### Import necessary libraries

In [1]:
import numpy as np
import pandas as pd

In [2]:
# Config Generation Sample Code.
# ⚠️ Only for experimentation on your side.
# ⚠️ Should be commented during submission.

# df = pd.DataFrame(data=[{'id':276633,
#                   'k':5,
#                   'dataset_file':'./Behance_appreciate_1M',
#                   'output_file':'./output.txt'}])
# df.to_csv('config.csv')

# df = pd.DataFrame(data=[{'id':10356,
#                   'k':5,
#                   'dataset_file':'./Behance_subsampled.txt',
#                   'output_file':'./output2.txt'}])
# df.to_csv('config.csv')

In [3]:
config = pd.read_csv('config.csv').iloc[0]

In [5]:
config

Unnamed: 0                             0
id                                 10356
k                                      5
dataset_file    ./Behance_subsampled.txt
output_file                ./output2.txt
Name: 0, dtype: object

In [4]:
user = config['id']
k_value = config['k']

### Read the Data

In [6]:
with open(config['dataset_file'], 'r') as inFile:
    appreciate_data = inFile.readlines()

In [ ]:
# your code here

### Initialize a dictionary to store the item_ids that a user likes

### Go through each line of the input file and construct the user_likes dictionary

In [7]:
user_likes = dict()
items = set()
users = set()

In [8]:
for line in appreciate_data:
    line = line.strip()
    
    user_id = int(line.split()[0])
    item_id = int(line.split()[1])

    items.add(item_id)
    users.add(user_id)

    if user_id not in user_likes:
        user_likes[user_id] = list()
    
    user_likes[user_id].append(item_id)

In [9]:
items = sorted(items)
items[0]
users = sorted(users)
users[0]
# users

10356

In [10]:
user_likes_matrix = [ [0 for i in range(len(items))] for i in range(len(list(user_likes.keys()))) ]

In [11]:
for user_idx in range(len(user_likes.keys())):
    for item_idx in range(len(user_likes_matrix[user_idx])):
        if items[item_idx] in user_likes[users[user_idx]]:
            user_likes_matrix[user_idx][item_idx] = 1

In [12]:
# getting latent features
import numpy as np
from sklearn.decomposition import TruncatedSVD

def get_latent_features(user_likes_matrix):
    epsilon = 1e-9
    n_latent_factors = 1000
    user_svd = TruncatedSVD(n_components = n_latent_factors)
    user_features = user_svd.fit_transform(user_likes_matrix) + epsilon

    return user_features


In [13]:
user_features = get_latent_features(user_likes_matrix).astype("float32")
# type(user_features)

### Use your choice of Approximate Nearest Neigbor after Collaborative Filtering to find nearest neighbors

In [14]:
import faiss                   # make faiss available

In [15]:
user_idx = 0
for i in range(len(users)):
    if users[i] == user: 
        user_idx = i
        break
query_features = [user_features[user_idx]]
query_features = np.array(query_features).astype("float32")

In [22]:
# your code here
def neighbors(user,user_features,query_features,k_value):
    """ returns an iterable object (like list or generator) """

    nlist = 5  # number of clusters
    quantiser = faiss.IndexFlatL2(1000)  
    index = faiss.IndexIVFFlat(quantiser, 1000, nlist,  faiss.METRIC_L2)
    # print(index.is_trained)
    index.train(user_features)
    # print(index.ntotal)
    index.add(user_features)                  # add vectors to the index
    # print(index.is_trained)
    # print(index.ntotal)

    index.nprobe = 2
    k = int(k_value + 1)                      # we want to see k+1 nearest neighbors
    D, I = index.search(query_features, k=k)     # actual search
    # print(I)                   # neighbors of the 5 first queries

    neighbors = []
    for user_idx in I[0]:
        # print(users[user_idx])
        if users[user_idx] == user: continue
        neighbors.append(users[user_idx])

    return neighbors

### Open the output file to write all the lines to the file

In [23]:
outFile = open(config['output_file'], 'w')

for n_user in neighbors(user,user_features,query_features,k_value):
    user_id = n_user
    for item_id in user_likes[user_id]:
        if item_id not in user_likes[user]:
            outFile.write(str(item_id) + ' ' + str(user_id) + '\n')

outFile.close()

False
0
True
1000


### Answer the following questions:

#### Q1. **Explain how your choice of library works**

Chosen library: FAISS

FAISS is a library developed by Facebook AI research that contains algorithms for efficient similarity search for vectors of any size, even those that might not fit in RAM. FAISS compares vectors with Euclidean (l2) distances and looks for vectors with the lowest l2 distance as the given query vector.

It is build around an index class that stores the set of vectors which the distance of which will be compared to the query vector. It provides a "search" function to search in this index with l2 vector comparision. FAISS essentially builds the index data structure in RAM and can then efficiently perform the operation: 

i = argmin_i ||x - x_i||

where there are a given set of vectors x_i of dimension d, and a new vector x is given. The search function is the computation of this argmin. There are different types of index structures which have their own tradeoffs with respect to search time, quality, memory used, and trainin time.

Using the distance metric, faiss first builds the Approximate Nearest Neighbours graph of the vectors. To make the search efficient, Faiss uses approximate non-exhaustive search on a Nearest-Neighbours graph. It also transforms the vectors to have a lower dimensionality. This preprocessing step in our case was to L2 normalize the vectors. The next step is inverted file indexing which partitions the vectors into similar clusters/centroids. So when our query vector is searched , it first calculates distance to nearest centroid of a cluster and only then the vectors of that cluster are accessed. This is followed by a last vector encoding step to further make index efficient. In all , faiss offers approximate search for a tiny reduction in accuracy for a very good performance in time and memory.


#### Q2. **Compare your choice of library with vanilla KNN.**
***Hint: Include Time Complexity, and explain the tradeoff with recall***

FAISS can perform vanilla KNN similar to what we have implemented in question 1 using the IndexFlatL2 data structure. In order to employ Approximate Nearest Neighbors as this question asked, we use the IndexIVFFlat data structure which takes a list in vectors that represent the database and performs Voronoi Tesselation in the n-dimensional vector space. This type of index requires a quantizer which is another index data structure (IndexFlatL2) which assigns vectors to Voronoi cells. Each cell is defined by centroid and by finding the nearest neighbour of the vector in the set of centroids the voronoi cell is found. During search, we can define how many neighbouring cells to probe for nearest neighbours from the nprobe parameter. This implementation leads to a much faster search speed. As per documentation, using approximation leads to a 10x increase in search speed with a 10% recall reduction.

The time complexity of vanilla KNN is O(n) for each query vector, n being the size of the dataset.

The library also allows ways to optimize the vector storage using another quantizer that is based on lossy compression techniues.


#### Q3. **Compare your choice of library with implementation of ScaNN, faiss and annoy.**
***Hint: Include Time Complexity, and explain the tradeoff with recall***

ScaNN: Scalable Nearest Neighbours is a library from Google Research that efficiently searched for similar vectors at scale. It follows the following steps :
    * Partitions the dataset while training to get the top partition in order to use during the scoring stage at query time. Implemented via a tree data structure. More leaves leads to higher quality of partioning with the drawback of longer time taken
    * Computing the distance from query vector to all datapoints in a partition while searching. Done either brute force which has the best accuracy or asymmetric hashing which works faster but leads to a drop in accuracy. There is also a quantized brute force that quantizes each dimensions datapoint. With large datasets this works much faster than brute force and doesn't have much change in accuracy
    * Rescoring the best k vector distances and recomputes the distances more accurately, then choosing top k

FAISS: supports differnt KNN algorithms with their own speed, memory, accuracy tradeoffs such as brute force, voronoi tesselation, product quantization etc by using a different index data structure

Annoy: implemented with a tree data structure. At each node in the tree, a random hyperplane is chosen which divides the space into two subspaces. A hyperplace is chosen by taking to points from a subset and taking the equidistant hyperplane from them. After doing this k times, you get a forest of trees. Depending on k, there is a precision-performance tradeoff.